In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [25]:
df = pd.read_csv(r'C:\Users\Kruti Agrawal\Desktop\Projects\vehicle_price_prediction\Vehicle Price Prediction\dataset.csv')

In [26]:
print(df.head())

                              name  \
0     2024 Jeep Wagoneer Series II   
1  2024 Jeep Grand Cherokee Laredo   
2         2024 GMC Yukon XL Denali   
3       2023 Dodge Durango Pursuit   
4            2024 RAM 3500 Laramie   

                                         description   make           model  \
0  \n      \n        Heated Leather Seats, Nav Sy...   Jeep        Wagoneer   
1  Al West is committed to offering every custome...   Jeep  Grand Cherokee   
2                                                NaN    GMC        Yukon XL   
3  White Knuckle Clearcoat 2023 Dodge Durango Pur...  Dodge         Durango   
4  \n      \n        2024 Ram 3500 Laramie Billet...    RAM            3500   

   year    price                                             engine  \
0  2024  74600.0                            24V GDI DOHC Twin Turbo   
1  2024  50170.0                                                OHV   
2  2024  96410.0  6.2L V-8 gasoline direct injection, variable v...   
3  2023  468

In [27]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1002 entries, 0 to 1001
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            1002 non-null   object 
 1   description     946 non-null    object 
 2   make            1002 non-null   object 
 3   model           1002 non-null   object 
 4   year            1002 non-null   int64  
 5   price           979 non-null    float64
 6   engine          1000 non-null   object 
 7   cylinders       897 non-null    float64
 8   fuel            995 non-null    object 
 9   mileage         968 non-null    float64
 10  transmission    1000 non-null   object 
 11  trim            1001 non-null   object 
 12  body            999 non-null    object 
 13  doors           995 non-null    float64
 14  exterior_color  997 non-null    object 
 15  interior_color  964 non-null    object 
 16  drivetrain      1002 non-null   object 
dtypes: float64(4), int64(1), object(1

In [28]:
print(df.isnull().sum())

name                0
description        56
make                0
model               0
year                0
price              23
engine              2
cylinders         105
fuel                7
mileage            34
transmission        2
trim                1
body                3
doors               7
exterior_color      5
interior_color     38
drivetrain          0
dtype: int64


In [29]:
df.dropna(inplace=True)

In [30]:
X = df.drop('price', axis=1)
y = df['price']

In [31]:
categorical_cols = ['make', 'model', 'fuel', 'transmission', 'trim', 'body', 'exterior_color', 'interior_color', 'drivetrain']
numerical_cols = ['year', 'cylinders', 'mileage', 'doors']

In [32]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)  
    ])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [35]:
X_train_dense = X_train.toarray()  
X_test_dense = X_test.toarray() 

In [36]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

In [37]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [38]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [39]:
history = model.fit(X_train_dense, y_train, validation_split=0.2, epochs=80, batch_size=32, callbacks=[early_stopping])

Epoch 1/80
15/15 [==============================] - 1s 7ms/step - loss: 2724596224.0000 - val_loss: 2568279296.0000
Epoch 2/80
15/15 [==============================] - 0s 3ms/step - loss: 2724372480.0000 - val_loss: 2567884544.0000
Epoch 3/80
15/15 [==============================] - 0s 2ms/step - loss: 2723571200.0000 - val_loss: 2566545664.0000
Epoch 4/80
15/15 [==============================] - 0s 3ms/step - loss: 2721030144.0000 - val_loss: 2562608896.0000
Epoch 5/80
15/15 [==============================] - 0s 3ms/step - loss: 2714210048.0000 - val_loss: 2552812800.0000
Epoch 6/80
15/15 [==============================] - 0s 3ms/step - loss: 2698338304.0000 - val_loss: 2531696128.0000
Epoch 7/80
15/15 [==============================] - 0s 3ms/step - loss: 2666163456.0000 - val_loss: 2491072512.0000
Epoch 8/80
15/15 [==============================] - 0s 3ms/step - loss: 2607074304.0000 - val_loss: 2420843264.0000
Epoch 9/80
15/15 [==============================] - 0s 3ms/step - loss: 

In [40]:
test_loss = model.evaluate(X_test_dense, y_test)
print(f'Test Loss: {test_loss}')

5/5 [==============================] - 0s 2ms/step - loss: 75823128.0000
Test Loss: 75823128.0


In [41]:
predictions = model.predict(X_test)

5/5 [==============================] - 0s 2ms/step


In [42]:
pred_df = pd.DataFrame({'Actual Price': y_test, 'Predicted Price': predictions.flatten()})
print(pred_df.head())

     Actual Price  Predicted Price
688       30870.0     39280.968750
492       54949.0     58166.414062
181       34330.0     33622.824219
339       51083.0     54051.296875
406       44604.0     48099.292969


In [43]:
model_path = 'C:/Users/Kruti Agrawal/Desktop/Projects/vehicle_price_prediction/model.h5'
model.save(model_path)

C:\Users\Kruti Agrawal\anaconda3\envs\new_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
preprocessor_path = 'C:/Users/Kruti Agrawal/Desktop/Projects/vehicle_price_prediction/preprocessor.pkl'
joblib.dump(preprocessor, preprocessor_path)

['C:/Users/Kruti Agrawal/Desktop/Projects/vehicle_price_prediction/preprocessor.pkl']